In [1]:
# Sample Data Run
from path import Path
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
data1 = Path('Resources/Presidential_96_2020.csv')
df1 = pd.read_csv(data1)
df1.head()

,year,state,DEMOCRAT_VOTES,LIBERTARIAN_VOTES,OTHER_VOTES,REPUBLICAN_VOTES,Winning_Party
0,1996,ALABAMA,662165,5290.0,97850.0,769044,REPUBLICAN
1,1996,ALASKA,80380,2276.0,36218.0,122746,REPUBLICAN
2,1996,ARIZONA,653288,14358.0,114686.0,622073,DEMOCRAT
3,1996,ARKANSAS,475171,3076.0,80599.0,325416,DEMOCRAT
4,1996,CALIFORNIA,5119835,73600.0,997653.0,3828381,DEMOCRAT


In [3]:
df1['Outcome'] = np.where(df1['Winning_Party']=='REPUBLICAN', 1, 0)
df1

,year,state,DEMOCRAT_VOTES,LIBERTARIAN_VOTES,OTHER_VOTES,REPUBLICAN_VOTES,Winning_Party,Outcome
0,1996,ALABAMA,662165,5290.0,97850.0,769044,REPUBLICAN,1
1,1996,ALASKA,80380,2276.0,36218.0,122746,REPUBLICAN,1
2,1996,ARIZONA,653288,14358.0,114686.0,622073,DEMOCRAT,0
3,1996,ARKANSAS,475171,3076.0,80599.0,325416,DEMOCRAT,0
4,1996,CALIFORNIA,5119835,73600.0,997653.0,3828381,DEMOCRAT,0
...,...,...,...,...,...,...,...,...
352,2020,VIRGINIA,2413568,64761.0,19765.0,1962430,DEMOCRAT,0
353,2020,WASHINGTON,2369612,80500.0,52868.0,1584651,DEMOCRAT,0
354,2020,WEST VIRGINIA,235984,10687.0,2599.0,545382,REPUBLICAN,1
355,2020,WISCONSIN,1630866,NaN,56991.0,1610184,DEMOCRAT,0


In [4]:
df1 = df1[df1.year >= 2000]
df1


,year,state,DEMOCRAT_VOTES,LIBERTARIAN_VOTES,OTHER_VOTES,REPUBLICAN_VOTES,Winning_Party,Outcome
51,2000,ALABAMA,692611,5893.0,26595.0,941173,REPUBLICAN,1
52,2000,ALASKA,79004,2636.0,36522.0,167398,REPUBLICAN,1
53,2000,ARIZONA,685341,5775.0,59248.0,781652,REPUBLICAN,1
54,2000,ARKANSAS,422768,2781.0,23292.0,472940,REPUBLICAN,1
55,2000,CALIFORNIA,5861203,45520.0,491670.0,4567429,DEMOCRAT,0
...,...,...,...,...,...,...,...,...
352,2020,VIRGINIA,2413568,64761.0,19765.0,1962430,DEMOCRAT,0
353,2020,WASHINGTON,2369612,80500.0,52868.0,1584651,DEMOCRAT,0
354,2020,WEST VIRGINIA,235984,10687.0,2599.0,545382,REPUBLICAN,1
355,2020,WISCONSIN,1630866,NaN,56991.0,1610184,DEMOCRAT,0


In [5]:
df1 = df1.rename(columns=str.upper)
df1

,YEAR,STATE,DEMOCRAT_VOTES,LIBERTARIAN_VOTES,OTHER_VOTES,REPUBLICAN_VOTES,WINNING_PARTY,OUTCOME
51,2000,ALABAMA,692611,5893.0,26595.0,941173,REPUBLICAN,1
52,2000,ALASKA,79004,2636.0,36522.0,167398,REPUBLICAN,1
53,2000,ARIZONA,685341,5775.0,59248.0,781652,REPUBLICAN,1
54,2000,ARKANSAS,422768,2781.0,23292.0,472940,REPUBLICAN,1
55,2000,CALIFORNIA,5861203,45520.0,491670.0,4567429,DEMOCRAT,0
...,...,...,...,...,...,...,...,...
352,2020,VIRGINIA,2413568,64761.0,19765.0,1962430,DEMOCRAT,0
353,2020,WASHINGTON,2369612,80500.0,52868.0,1584651,DEMOCRAT,0
354,2020,WEST VIRGINIA,235984,10687.0,2599.0,545382,REPUBLICAN,1
355,2020,WISCONSIN,1630866,NaN,56991.0,1610184,DEMOCRAT,0


In [6]:
df1_final = df1[['YEAR','STATE','OUTCOME']]
df1_final

,YEAR,STATE,OUTCOME
51,2000,ALABAMA,1
52,2000,ALASKA,1
53,2000,ARIZONA,1
54,2000,ARKANSAS,1
55,2000,CALIFORNIA,0
...,...,...,...
352,2020,VIRGINIA,0
353,2020,WASHINGTON,0
354,2020,WEST VIRGINIA,1
355,2020,WISCONSIN,0


In [7]:
data2 = Path('Resources/CENSUS_00_19.csv')
df2 = pd.read_csv(data2)
df2['STATE_NAME'] = df2['STATE_NAME'].str.upper()
df2

,YEAR,STATE_NAME,STATE_NUMBER,REGION,DIVISION,TOTAL_POPULATION,RACE_WHITE,RACE_BLACK,RACE_OTHER,AGE_0_TO_19,AGE_20_TO_39,AGE_40_TO_59,AGE_60_TO_79,AGE_80_AND_OVER,ORIGIN_HISPANIC,ORIGIN_NOT_HISPANIC,SEX_FEMALE,SEX_MALE
0,2000,ALABAMA,1,South,East South Central,4447207,3195561,1158988,92658,1256218,1250228,1170854,624964,144943,75821,4371386,2300647,2146560
1,2000,ALASKA,2,West,Pacific,626933,446431,22909,157593,208114,185095,180695,46700,6329,25852,601081,302821,324112
2,2000,ARIZONA,4,West,Mountain,5130247,4518143,168952,443152,1518175,1498202,1242614,711472,159784,1295616,3834631,2569382,2560865
3,2000,ARKANSAS,5,South,West South Central,2673293,2183097,420516,69680,760477,734755,686669,392228,99164,86862,2586431,1368647,1304646
4,2000,CALIFORNIA,6,West,Pacific,33871653,26365296,2381132,5125225,10234576,10425088,8469482,3814015,928492,10966547,22905106,16996778,16874875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,2019,VIRGINIA,51,South,South Atlantic,8535519,5922648,1696911,915960,2087426,2343713,2215430,1579134,309816,834422,7701097,4335262,4200257
1016,2019,WASHINGTON,53,West,Pacific,7614893,5977696,331924,1305273,1840306,2203448,1890236,1417638,263265,991721,6623172,3802910,3811983
1017,2019,WEST VIRGINIA,54,South,South Atlantic,1792147,1675122,64610,52415,402473,431297,461667,412728,83982,31162,1760985,904377,887770
1018,2019,WISCONSIN,55,Midwest,East North Central,5822434,5067781,390543,364110,1422095,1505295,1477023,1170814,247207,413208,5409226,2925225,2897209


In [8]:
df2['PCT_WHITE'] = df2.RACE_WHITE / df2.TOTAL_POPULATION
df2['PCT_BLACK'] = df2.RACE_BLACK / df2.TOTAL_POPULATION
df2['PCT_HISPANIC'] = df2.ORIGIN_HISPANIC / df2.TOTAL_POPULATION
df2['PCT_OVER_60'] = (df2.AGE_60_TO_79 + df2.AGE_80_AND_OVER) / df2.TOTAL_POPULATION
df2['PCT_UNDER_40'] = (df2.AGE_0_TO_19 + df2.AGE_20_TO_39) / df2.TOTAL_POPULATION

df2

,YEAR,STATE_NAME,STATE_NUMBER,REGION,DIVISION,TOTAL_POPULATION,RACE_WHITE,RACE_BLACK,RACE_OTHER,AGE_0_TO_19,...,AGE_80_AND_OVER,ORIGIN_HISPANIC,ORIGIN_NOT_HISPANIC,SEX_FEMALE,SEX_MALE,PCT_WHITE,PCT_BLACK,PCT_HISPANIC,PCT_OVER_60,PCT_UNDER_40
0,2000,ALABAMA,1,South,East South Central,4447207,3195561,1158988,92658,1256218,...,144943,75821,4371386,2300647,2146560,0.718555,0.260610,0.017049,0.173121,0.563600
1,2000,ALASKA,2,West,Pacific,626933,446431,22909,157593,208114,...,6329,25852,601081,302821,324112,0.712087,0.036541,0.041236,0.084585,0.627195
2,2000,ARIZONA,4,West,Mountain,5130247,4518143,168952,443152,1518175,...,159784,1295616,3834631,2569382,2560865,0.880687,0.032933,0.252545,0.169827,0.587959
3,2000,ARKANSAS,5,South,West South Central,2673293,2183097,420516,69680,760477,...,99164,86862,2586431,1368647,1304646,0.816632,0.157303,0.032493,0.183815,0.559322
4,2000,CALIFORNIA,6,West,Pacific,33871653,26365296,2381132,5125225,10234576,...,928492,10966547,22905106,16996778,16874875,0.778388,0.070299,0.323768,0.140014,0.609940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,2019,VIRGINIA,51,South,South Atlantic,8535519,5922648,1696911,915960,2087426,...,309816,834422,7701097,4335262,4200257,0.693883,0.198806,0.097759,0.221305,0.519141
1016,2019,WASHINGTON,53,West,Pacific,7614893,5977696,331924,1305273,1840306,...,263265,991721,6623172,3802910,3811983,0.785001,0.043589,0.130234,0.220739,0.531032
1017,2019,WEST VIRGINIA,54,South,South Atlantic,1792147,1675122,64610,52415,402473,...,83982,31162,1760985,904377,887770,0.934701,0.036052,0.017388,0.277159,0.465235
1018,2019,WISCONSIN,55,Midwest,East North Central,5822434,5067781,390543,364110,1422095,...,247207,413208,5409226,2925225,2897209,0.870389,0.067076,0.070968,0.243544,0.502778


In [9]:
df2.loc[df2['YEAR'] == 2019, 'YEAR'] = 2020
df2


,YEAR,STATE_NAME,STATE_NUMBER,REGION,DIVISION,TOTAL_POPULATION,RACE_WHITE,RACE_BLACK,RACE_OTHER,AGE_0_TO_19,...,AGE_80_AND_OVER,ORIGIN_HISPANIC,ORIGIN_NOT_HISPANIC,SEX_FEMALE,SEX_MALE,PCT_WHITE,PCT_BLACK,PCT_HISPANIC,PCT_OVER_60,PCT_UNDER_40
0,2000,ALABAMA,1,South,East South Central,4447207,3195561,1158988,92658,1256218,...,144943,75821,4371386,2300647,2146560,0.718555,0.260610,0.017049,0.173121,0.563600
1,2000,ALASKA,2,West,Pacific,626933,446431,22909,157593,208114,...,6329,25852,601081,302821,324112,0.712087,0.036541,0.041236,0.084585,0.627195
2,2000,ARIZONA,4,West,Mountain,5130247,4518143,168952,443152,1518175,...,159784,1295616,3834631,2569382,2560865,0.880687,0.032933,0.252545,0.169827,0.587959
3,2000,ARKANSAS,5,South,West South Central,2673293,2183097,420516,69680,760477,...,99164,86862,2586431,1368647,1304646,0.816632,0.157303,0.032493,0.183815,0.559322
4,2000,CALIFORNIA,6,West,Pacific,33871653,26365296,2381132,5125225,10234576,...,928492,10966547,22905106,16996778,16874875,0.778388,0.070299,0.323768,0.140014,0.609940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,2020,VIRGINIA,51,South,South Atlantic,8535519,5922648,1696911,915960,2087426,...,309816,834422,7701097,4335262,4200257,0.693883,0.198806,0.097759,0.221305,0.519141
1016,2020,WASHINGTON,53,West,Pacific,7614893,5977696,331924,1305273,1840306,...,263265,991721,6623172,3802910,3811983,0.785001,0.043589,0.130234,0.220739,0.531032
1017,2020,WEST VIRGINIA,54,South,South Atlantic,1792147,1675122,64610,52415,402473,...,83982,31162,1760985,904377,887770,0.934701,0.036052,0.017388,0.277159,0.465235
1018,2020,WISCONSIN,55,Midwest,East North Central,5822434,5067781,390543,364110,1422095,...,247207,413208,5409226,2925225,2897209,0.870389,0.067076,0.070968,0.243544,0.502778


In [10]:
df2_final = df2[['YEAR','STATE_NAME','REGION','PCT_WHITE','PCT_BLACK','PCT_HISPANIC','PCT_OVER_60','PCT_UNDER_40']]
df2_final

,YEAR,STATE_NAME,REGION,PCT_WHITE,PCT_BLACK,PCT_HISPANIC,PCT_OVER_60,PCT_UNDER_40
0,2000,ALABAMA,South,0.718555,0.260610,0.017049,0.173121,0.563600
1,2000,ALASKA,West,0.712087,0.036541,0.041236,0.084585,0.627195
2,2000,ARIZONA,West,0.880687,0.032933,0.252545,0.169827,0.587959
3,2000,ARKANSAS,South,0.816632,0.157303,0.032493,0.183815,0.559322
4,2000,CALIFORNIA,West,0.778388,0.070299,0.323768,0.140014,0.609940
...,...,...,...,...,...,...,...,...
1015,2020,VIRGINIA,South,0.693883,0.198806,0.097759,0.221305,0.519141
1016,2020,WASHINGTON,West,0.785001,0.043589,0.130234,0.220739,0.531032
1017,2020,WEST VIRGINIA,South,0.934701,0.036052,0.017388,0.277159,0.465235
1018,2020,WISCONSIN,Midwest,0.870389,0.067076,0.070968,0.243544,0.502778


In [11]:
df2_final2 = pd.get_dummies(data=df2_final, columns=['REGION'])
df2_final2

,YEAR,STATE_NAME,PCT_WHITE,PCT_BLACK,PCT_HISPANIC,PCT_OVER_60,PCT_UNDER_40,REGION_Midwest,REGION_Northeast,REGION_South,REGION_West
0,2000,ALABAMA,0.718555,0.260610,0.017049,0.173121,0.563600,0,0,1,0
1,2000,ALASKA,0.712087,0.036541,0.041236,0.084585,0.627195,0,0,0,1
2,2000,ARIZONA,0.880687,0.032933,0.252545,0.169827,0.587959,0,0,0,1
3,2000,ARKANSAS,0.816632,0.157303,0.032493,0.183815,0.559322,0,0,1,0
4,2000,CALIFORNIA,0.778388,0.070299,0.323768,0.140014,0.609940,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1015,2020,VIRGINIA,0.693883,0.198806,0.097759,0.221305,0.519141,0,0,1,0
1016,2020,WASHINGTON,0.785001,0.043589,0.130234,0.220739,0.531032,0,0,0,1
1017,2020,WEST VIRGINIA,0.934701,0.036052,0.017388,0.277159,0.465235,0,0,1,0
1018,2020,WISCONSIN,0.870389,0.067076,0.070968,0.243544,0.502778,1,0,0,0


In [12]:
merged_df = pd.merge(df1_final, df2_final2, how='inner', left_on=['YEAR','STATE'], right_on = ['YEAR','STATE_NAME'])
merged_df

,YEAR,STATE,OUTCOME,STATE_NAME,PCT_WHITE,PCT_BLACK,PCT_HISPANIC,PCT_OVER_60,PCT_UNDER_40,REGION_Midwest,REGION_Northeast,REGION_South,REGION_West
0,2000,ALABAMA,1,ALABAMA,0.718555,0.260610,0.017049,0.173121,0.563600,0,0,1,0
1,2000,ALASKA,1,ALASKA,0.712087,0.036541,0.041236,0.084585,0.627195,0,0,0,1
2,2000,ARIZONA,1,ARIZONA,0.880687,0.032933,0.252545,0.169827,0.587959,0,0,0,1
3,2000,ARKANSAS,1,ARKANSAS,0.816632,0.157303,0.032493,0.183815,0.559322,0,0,1,0
4,2000,CALIFORNIA,0,CALIFORNIA,0.778388,0.070299,0.323768,0.140014,0.609940,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,2020,VIRGINIA,0,VIRGINIA,0.693883,0.198806,0.097759,0.221305,0.519141,0,0,1,0
302,2020,WASHINGTON,0,WASHINGTON,0.785001,0.043589,0.130234,0.220739,0.531032,0,0,0,1
303,2020,WEST VIRGINIA,1,WEST VIRGINIA,0.934701,0.036052,0.017388,0.277159,0.465235,0,0,1,0
304,2020,WISCONSIN,0,WISCONSIN,0.870389,0.067076,0.070968,0.243544,0.502778,1,0,0,0


In [13]:
data3 = Path('Resources/PAST_5_ELECTION_VOTING.csv')
df3 = pd.read_csv(data3)

df3

,YEAR,STATE,state_po,state_fips,state_cen,state_ic,office,candidate,party_detailed,writein,candidatevotes,totalvotes,version,notes,party_simplified,Outcome,PAST_5_ELECTION_VOTING
0,1976,ALABAMA,AL,1,63,41,US PRESIDENT,"CARTER, JIMMY",DEMOCRAT,False,659170,1182850,20210113,NaN,DEMOCRAT,0,0.0
1,1980,ALABAMA,AL,1,63,41,US PRESIDENT,"REAGAN, RONALD",REPUBLICAN,False,654192,1341929,20210113,NaN,REPUBLICAN,1,0.0
2,1984,ALABAMA,AL,1,63,41,US PRESIDENT,"REAGAN, RONALD",REPUBLICAN,False,872849,1441713,20210113,NaN,REPUBLICAN,1,0.0
3,1988,ALABAMA,AL,1,63,41,US PRESIDENT,"BUSH, GEORGE H.W.",REPUBLICAN,False,815576,1378476,20210113,NaN,REPUBLICAN,1,0.0
4,1992,ALABAMA,AL,1,63,41,US PRESIDENT,"BUSH, GEORGE H.W.",REPUBLICAN,False,804283,1688060,20210113,NaN,REPUBLICAN,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,2004,WYOMING,WY,56,83,68,US PRESIDENT,"BUSH, GEORGE W.",REPUBLICAN,False,167629,243861,20210113,NaN,REPUBLICAN,1,1.0
608,2008,WYOMING,WY,56,83,68,US PRESIDENT,"MCCAIN, JOHN",REPUBLICAN,False,164958,254904,20210113,NaN,REPUBLICAN,1,1.0
609,2012,WYOMING,WY,56,83,68,US PRESIDENT,"ROMNEY, MITT",REPUBLICAN,False,170962,250701,20210113,NaN,REPUBLICAN,1,1.0
610,2016,WYOMING,WY,56,83,68,US PRESIDENT,"TRUMP, DONALD J.",REPUBLICAN,False,174419,258788,20210113,NaN,REPUBLICAN,1,1.0


In [14]:
df3 = df3[df3.YEAR >= 2000]
df3

,YEAR,STATE,state_po,state_fips,state_cen,state_ic,office,candidate,party_detailed,writein,candidatevotes,totalvotes,version,notes,party_simplified,Outcome,PAST_5_ELECTION_VOTING
6,2000,ALABAMA,AL,1,63,41,US PRESIDENT,"BUSH, GEORGE W.",REPUBLICAN,False,941173,1666272,20210113,NaN,REPUBLICAN,1,1.0
7,2004,ALABAMA,AL,1,63,41,US PRESIDENT,"BUSH, GEORGE W.",REPUBLICAN,False,1176394,1883415,20210113,NaN,REPUBLICAN,1,1.0
8,2008,ALABAMA,AL,1,63,41,US PRESIDENT,"MCCAIN, JOHN",REPUBLICAN,False,1266546,2099819,20210113,NaN,REPUBLICAN,1,1.0
9,2012,ALABAMA,AL,1,63,41,US PRESIDENT,"ROMNEY, MITT",REPUBLICAN,False,1255925,2074338,20210113,NaN,REPUBLICAN,1,1.0
10,2016,ALABAMA,AL,1,63,41,US PRESIDENT,"TRUMP, DONALD J.",REPUBLICAN,False,1318255,2123372,20210113,NaN,REPUBLICAN,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,2004,WYOMING,WY,56,83,68,US PRESIDENT,"BUSH, GEORGE W.",REPUBLICAN,False,167629,243861,20210113,NaN,REPUBLICAN,1,1.0
608,2008,WYOMING,WY,56,83,68,US PRESIDENT,"MCCAIN, JOHN",REPUBLICAN,False,164958,254904,20210113,NaN,REPUBLICAN,1,1.0
609,2012,WYOMING,WY,56,83,68,US PRESIDENT,"ROMNEY, MITT",REPUBLICAN,False,170962,250701,20210113,NaN,REPUBLICAN,1,1.0
610,2016,WYOMING,WY,56,83,68,US PRESIDENT,"TRUMP, DONALD J.",REPUBLICAN,False,174419,258788,20210113,NaN,REPUBLICAN,1,1.0


In [15]:
df3_final = df3[['YEAR','STATE','PAST_5_ELECTION_VOTING']]

In [16]:
merged_df = pd.merge(merged_df, df3_final, how='inner', left_on=['YEAR','STATE'], right_on = ['YEAR','STATE'])

merged_df = merged_df.drop(columns=['STATE_NAME','PCT_OVER_60','PCT_UNDER_40'])
merged_df.set_index(['YEAR','STATE'])
merged_df

,YEAR,STATE,OUTCOME,PCT_WHITE,PCT_BLACK,PCT_HISPANIC,REGION_Midwest,REGION_Northeast,REGION_South,REGION_West,PAST_5_ELECTION_VOTING
0,2000,ALABAMA,1,0.718555,0.260610,0.017049,0,0,1,0,1.0
1,2000,ALASKA,1,0.712087,0.036541,0.041236,0,0,0,1,1.0
2,2000,ARIZONA,1,0.880687,0.032933,0.252545,0,0,0,1,0.8
3,2000,ARKANSAS,1,0.816632,0.157303,0.032493,0,0,1,0,0.6
4,2000,CALIFORNIA,0,0.778388,0.070299,0.323768,0,0,0,1,0.6
...,...,...,...,...,...,...,...,...,...,...,...
301,2020,VIRGINIA,0,0.693883,0.198806,0.097759,0,0,1,0,0.4
302,2020,WASHINGTON,0,0.785001,0.043589,0.130234,0,0,0,1,0.0
303,2020,WEST VIRGINIA,1,0.934701,0.036052,0.017388,0,0,1,0,1.0
304,2020,WISCONSIN,0,0.870389,0.067076,0.070968,1,0,0,0,0.2


In [22]:
data2024 = Path('Resources/PROJECTIONS_2024.csv')
df2024 = pd.read_csv(data2024)
df2024['STATE'] = df2024['STATE'].str.upper()
df2024

,STATE,PCT_WHITE,PCT_BLACK,PCT_HISPANIC,REGION,PAST_5_ELECTION_VOTING,OUTCOME_2020
0,ALABAMA,0.682357,0.270320,0.055573,South,1.0,1
1,ALASKA,0.636881,0.036990,0.082088,West,1.0,1
2,ARIZONA,0.807592,0.057454,0.338804,West,0.8,0
3,ARKANSAS,0.783003,0.156023,0.094060,South,1.0,1
4,CALIFORNIA,0.701734,0.062491,0.420171,West,0.0,0
5,COLORADO,0.857902,0.047395,0.234161,West,0.2,0
6,CONNECTICUT,0.779633,0.126900,0.188522,Northeast,0.0,0
7,DELAWARE,0.668456,0.242029,0.112310,South,0.0,0
8,DISTRICT OF COLUMBIA,0.499261,0.408816,0.122078,South,0.0,0
9,FLORIDA,0.756968,0.176690,0.292463,South,0.6,1


In [23]:
df2024_final = pd.get_dummies(data=df2024, columns=['REGION']).set_index('STATE')

df2024_final = df2024_final[['OUTCOME_2020','PCT_WHITE','PCT_BLACK','PCT_HISPANIC','REGION_Midwest','REGION_Northeast','REGION_South','REGION_West','PAST_5_ELECTION_VOTING']]

df2024_final

,OUTCOME_2020,PCT_WHITE,PCT_BLACK,PCT_HISPANIC,REGION_Midwest,REGION_Northeast,REGION_South,REGION_West,PAST_5_ELECTION_VOTING
STATE,,,,,,,,,
ALABAMA,1,0.682357,0.270320,0.055573,0,0,1,0,1.0
ALASKA,1,0.636881,0.036990,0.082088,0,0,0,1,1.0
ARIZONA,0,0.807592,0.057454,0.338804,0,0,0,1,0.8
ARKANSAS,1,0.783003,0.156023,0.094060,0,0,1,0,1.0
CALIFORNIA,0,0.701734,0.062491,0.420171,0,0,0,1,0.0
COLORADO,0,0.857902,0.047395,0.234161,0,0,0,1,0.2
CONNECTICUT,0,0.779633,0.126900,0.188522,0,1,0,0,0.0
DELAWARE,0,0.668456,0.242029,0.112310,0,0,1,0,0.0
DISTRICT OF COLUMBIA,0,0.499261,0.408816,0.122078,0,0,1,0,0.0


In [24]:
y_train = merged_df.OUTCOME
y_test = df2024_final.OUTCOME_2020
X_train = merged_df.drop(columns=['YEAR','STATE','OUTCOME'])
X_test = df2024_final.drop(columns=['OUTCOME_2020'])
X_train

,PCT_WHITE,PCT_BLACK,PCT_HISPANIC,REGION_Midwest,REGION_Northeast,REGION_South,REGION_West,PAST_5_ELECTION_VOTING
0,0.718555,0.260610,0.017049,0,0,1,0,1.0
1,0.712087,0.036541,0.041236,0,0,0,1,1.0
2,0.880687,0.032933,0.252545,0,0,0,1,0.8
3,0.816632,0.157303,0.032493,0,0,1,0,0.6
4,0.778388,0.070299,0.323768,0,0,0,1,0.6
...,...,...,...,...,...,...,...,...
301,0.693883,0.198806,0.097759,0,0,1,0,0.4
302,0.785001,0.043589,0.130234,0,0,0,1,0.0
303,0.934701,0.036052,0.017388,0,0,1,0,1.0
304,0.870389,0.067076,0.070968,1,0,0,0,0.2


In [25]:
classifier = LogisticRegression(solver='lbfgs',
                                random_state=1)

In [26]:
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [30]:
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction 2024": y_pred, "2020 Actual": y_test})
results

,Prediction 2024,2020 Actual
STATE,,
ALABAMA,1,1
ALASKA,1,1
ARIZONA,1,0
ARKANSAS,1,1
CALIFORNIA,0,0
COLORADO,0,0
CONNECTICUT,0,0
DELAWARE,0,0
DISTRICT OF COLUMBIA,0,0


In [31]:
results.to_csv("2024_predictions.csv")